In [1]:
import numpy as np
import torch
import scanpy as sc
import torch.nn.functional as F
from configure import get_default_config
from model_brca import Model
from tools import normalize_type
import random
from sklearn.model_selection import KFold

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
dataset = {
    0: "dro",
    1: "MERFISH",
    2: "STARmap",
    4: "BRCA",
    5: "ELSE",
}
data_name = dataset[4]
shuffle = False
random.seed(48)
ktimes = 10
cv = KFold(n_splits=ktimes, shuffle=False)

In [2]:
def filter_with_overlap_gene(adata, adata_sc):
    # remove all-zero-valued genes
    # sc.pp.filter_genes(adata, min_cells=1)
    # sc.pp.filter_genes(adata_sc, min_cells=1)
    if 'highly_variable' not in adata.var.keys():
        raise ValueError("'highly_variable' are not existed in adata!")
    else:
        adata = adata[:, adata.var['highly_variable']]

    if 'highly_variable' not in adata_sc.var.keys():
        raise ValueError("'highly_variable' are not existed in adata_sc!")
    else:
        adata_sc = adata_sc[:, adata_sc.var['highly_variable']]

        # Refine `marker_genes` so that they are shared by both adatas
    genes = list(set(adata.var.index) & set(adata_sc.var.index))
    genes.sort()
    print('Number of overlap genes:', len(genes))
    adata.uns["overlap_genes"] = genes
    adata_sc.uns["overlap_genes"] = genes

    adata = adata[:, genes]
    adata_sc = adata_sc[:, genes]

    return adata, adata_sc

In [3]:
rna_path = 'data/BRCA/scRNA.h5ad'
adata_sc = sc.read(rna_path)
adata_sc.var_names_make_unique()
sc.pp.highly_variable_genes(adata_sc, flavor="seurat_v3", n_top_genes=3000, inplace=True, subset=True)
sc.pp.normalize_total(adata_sc, target_sum=1e4)
sc.pp.log1p(adata_sc)
# load BRCA ST data
adata_st = sc.read_visium(path=r"./data/BRCA/",
                               count_file="filtered_feature_bc_matrix.h5", library_id="BRCA",
                               load_images=True, source_image_path="/spatial/")
adata_st.var_names_make_unique()
sc.pp.highly_variable_genes(adata_st, flavor="seurat_v3", n_top_genes=3000, inplace=True, subset=True)
sc.pp.normalize_total(adata_st, target_sum=1e4)
sc.pp.log1p(adata_st)
adata_st, adata_sc = filter_with_overlap_gene(adata_st, adata_sc)
atlas_genes = adata_st.uns["overlap_genes"]
x1_cell, x2_rna = adata_st.X, adata_sc.X
x1_cell = x1_cell.todense()

C:\ProgramData\Anaconda3\envs\torch1.10\lib\site-packages\scanpy\preprocessing\_highly_variable_genes.py:64: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  UserWarning,
C:\ProgramData\Anaconda3\envs\torch1.10\lib\site-packages\anndata\_core\anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


Number of overlap genes: 921


C:\ProgramData\Anaconda3\envs\torch1.10\lib\site-packages\anndata\compat\_overloaded_dict.py:106: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  self.data[key] = value


In [4]:
def pearsonr(x, y):
    mean_x = torch.mean(x)
    mean_y = torch.mean(y)
    xm = x.sub(mean_x)
    ym = y.sub(mean_y)
    r_num = xm.dot(ym)
    r_den = torch.norm(xm, 2) * torch.norm(ym, 2)
    r_val = r_num / (r_den + 1e-8)
    r_val = torch.nan_to_num(r_val,nan=-1)
    return r_val

def correlationMetric(x, y):
    corr = 0
    for idx in range(x.size(1)):
        corr += pearsonr(x[:,idx], y[:,idx])
    corr /= (idx + 1)
    return (1 - corr).mean()

In [5]:
class MinMaxScaler_torch:
    def __init__(self, feature_range=(0, 1)):
        self.min_val = None
        self.max_val = None
        self.feature_range = feature_range

    def fit(self, data):
        self.min_val, _ = torch.min(data, dim=0)
        self.max_val, _ = torch.max(data, dim=0)

    def transform(self, data):
        data_minmax = (data - self.min_val) / (self.max_val - self.min_val)
        data_minmax = data_minmax * (self.feature_range[1] - self.feature_range[0]) + self.feature_range[0]
        return data_minmax

    def inverse_transform(self, data_minmax):
        data = (data_minmax - self.feature_range[0]) / (self.feature_range[1] - self.feature_range[0])
        data = data * (self.max_val - self.min_val) + self.min_val
        return data

In [6]:
def train(model, optimizer, x1, x2, config, model_path):
    print('\n===========> Training... <===========')
    model.train()
    if not isinstance(x1, torch.Tensor):
        x1 = torch.from_numpy(x1).to(device)
    if not isinstance(x2, torch.Tensor):
        x2 = torch.from_numpy(x2).to(device)
    for epoch in range(config['pretrain_epochs']):
        x1_hat = model(x2)
        mask_zero = x1 != 0
        x1_masked = torch.masked_select(x1, mask_zero)
        x1_masked_pred = torch.masked_select(x1_hat, mask_zero)
        loss_ae = F.mse_loss(x1, x1_hat, reduction='mean')
        loss_mask_ae = F.mse_loss(x1_masked, x1_masked_pred, reduction='mean')
        corrloss = correlationMetric(x1_hat, x1)
        loss = loss_ae + 0.001 * loss_mask_ae + 0.01 * corrloss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % 10 == 0:
            print ('train epoch %d: loss_ae: %.6f loss_mask_ae: %.6f loss_corr: %.6f' \
                   % (epoch, loss.item(), loss_mask_ae.item(), corrloss.item()))
    torch.save(model.state_dict(), model_path)


In [7]:
cnts = 0
for train_idx, test_idx in cv.split(atlas_genes):
    x1_train_cell = x1_cell[:, train_idx]
    x1_test_cell = x1_cell[:, test_idx]
    x2_train_cell = x2_rna[:, train_idx]
    x2_test_cell = x2_rna[:, test_idx]
    print(x1_cell.shape, x2_rna.shape)
    x1 = x1_train_cell.astype(np.float32)
    x2 = x2_train_cell.astype(np.float32)
    print (x1.max(), x1.min(), x2.max(), x2.min())
    count_zerox1 = np.sum(x1 == 0)
    count_zerox2 = np.sum(x2 == 0)
    print('Before Norm ST:', float(count_zerox1) / float(x1.shape[0] * x1.shape[1]), 'Before Norm RNA:',
          float(count_zerox2) / float(x2.shape[0] * x2.shape[1]))
    x1, x2 = normalize_type(x1, x2, type='all')
    print ('Xrna shape:', x2.shape,  'Xst shape:', x1.shape)
    count_zerox1 = np.sum(x1 == 0)
    count_zerox2 = np.sum(x2 == 0)
    print ('ST:', float(count_zerox1) / float(x1.shape[0]*x1.shape[1]), 'RNA:', float(count_zerox2) / float(x2.shape[0]*x2.shape[1]))
    print (x1.max(), x1.min(), x1.mean(), x2.max(), x2.min(), x2.mean())
    config = get_default_config(data_name)
    config['num_sample1'] = x1.shape[0]
    config['num_sample2'] = x2.shape[0]
    train_path = "train/%s_%d.pkl" % (data_name, cnts)
    print (train_path)
    tmp_dims = x1.shape[1]
    model = Model(config)
    model.to(device)
    optimizer_pre = torch.optim.Adam(model.parameters(), lr=config['pre_lr'])
    train(model, optimizer_pre, x1, x2, config, train_path)
    print("Finished " + str(cnts) + ' Times Training and Testing...')
    cnts += 1


(3798, 921) (45647, 921)
8.3757105 0.0 6.1131206 0.0
Before Norm ST: 0.6720480268028176 Before Norm RNA: 0.7785909122610616


C:\ProgramData\Anaconda3\envs\torch1.10\lib\site-packages\sklearn\utils\validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
C:\ProgramData\Anaconda3\envs\torch1.10\lib\site-packages\sklearn\utils\validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


Xrna shape: (45647, 828) Xst shape: (3798, 828)
ST: 0.6720483447937257 RNA: 0.7785909122610616
1.0 0.0 0.1413657 1.0 0.0 0.115089804
train/BRCA_0.pkl

===========> Training... <===========
train epoch 0: loss_ae: 0.081008 loss_mask_ae: 0.215522 loss_corr: 1.004733
train epoch 10: loss_ae: 0.063611 loss_mask_ae: 0.143707 loss_corr: 0.971637
train epoch 20: loss_ae: 0.053623 loss_mask_ae: 0.076622 loss_corr: 0.920570
train epoch 30: loss_ae: 0.049898 loss_mask_ae: 0.071980 loss_corr: 0.887965
train epoch 40: loss_ae: 0.045990 loss_mask_ae: 0.070037 loss_corr: 0.876791
train epoch 50: loss_ae: 0.046238 loss_mask_ae: 0.056039 loss_corr: 0.874176
train epoch 60: loss_ae: 0.040302 loss_mask_ae: 0.076589 loss_corr: 0.868123
train epoch 70: loss_ae: 0.037731 loss_mask_ae: 0.067067 loss_corr: 0.856886
train epoch 80: loss_ae: 0.036295 loss_mask_ae: 0.061961 loss_corr: 0.850242
train epoch 90: loss_ae: 0.035487 loss_mask_ae: 0.053463 loss_corr: 0.845099
Finished 0 Times Training and Testing...
(

C:\ProgramData\Anaconda3\envs\torch1.10\lib\site-packages\sklearn\utils\validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
C:\ProgramData\Anaconda3\envs\torch1.10\lib\site-packages\sklearn\utils\validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


Xrna shape: (45647, 829) Xst shape: (3798, 829)
ST: 0.6695070289676937 RNA: 0.7782712530729932
1.0 0.0 0.14286344 1.0 0.0 0.114488915
train/BRCA_1.pkl

===========> Training... <===========
train epoch 0: loss_ae: 0.081783 loss_mask_ae: 0.216400 loss_corr: 0.998322
train epoch 10: loss_ae: 0.072879 loss_mask_ae: 0.177274 loss_corr: 0.988577
train epoch 20: loss_ae: 0.054041 loss_mask_ae: 0.114291 loss_corr: 0.929493
train epoch 30: loss_ae: 0.049813 loss_mask_ae: 0.071088 loss_corr: 0.889236
train epoch 40: loss_ae: 0.047202 loss_mask_ae: 0.067792 loss_corr: 0.876503
train epoch 50: loss_ae: 0.042510 loss_mask_ae: 0.066684 loss_corr: 0.874758
train epoch 60: loss_ae: 0.038970 loss_mask_ae: 0.068585 loss_corr: 0.870011
train epoch 70: loss_ae: 0.039218 loss_mask_ae: 0.076920 loss_corr: 0.861308
train epoch 80: loss_ae: 0.037091 loss_mask_ae: 0.050715 loss_corr: 0.855245
train epoch 90: loss_ae: 0.035787 loss_mask_ae: 0.051967 loss_corr: 0.849791
Finished 1 Times Training and Testing...


...

ST: 0.6715933914808823 RNA: 0.7811551872484086
1.0 0.0 0.14178143 1.0 0.0 0.11343517
train/BRCA_7.pkl

===========> Training... <===========
train epoch 0: loss_ae: 0.081134 loss_mask_ae: 0.215777 loss_corr: 0.999044
train epoch 10: loss_ae: 0.064084 loss_mask_ae: 0.147117 loss_corr: 0.968628
train epoch 20: loss_ae: 0.050987 loss_mask_ae: 0.101794 loss_corr: 0.909930
train epoch 30: loss_ae: 0.053585 loss_mask_ae: 0.121723 loss_corr: 0.887206
train epoch 40: loss_ae: 0.048104 loss_mask_ae: 0.097077 loss_corr: 0.877878
train epoch 50: loss_ae: 0.043268 loss_mask_ae: 0.068906 loss_corr: 0.874849
train epoch 60: loss_ae: 0.040315 loss_mask_ae: 0.062267 loss_corr: 0.875507
train epoch 70: loss_ae: 0.037721 loss_mask_ae: 0.065606 loss_corr: 0.868804
train epoch 80: loss_ae: 0.036669 loss_mask_ae: 0.064098 loss_corr: 0.860343
train epoch 90: loss_ae: 0.035556 loss_mask_ae: 0.054339 loss_corr: 0.851740
Finished 7 Times Training and Testing...
(3798, 921) (45647, 921)
8.3757105 0.0 6.1131206 

C:\ProgramData\Anaconda3\envs\torch1.10\lib\site-packages\sklearn\utils\validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
C:\ProgramData\Anaconda3\envs\torch1.10\lib\site-packages\sklearn\utils\validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


ST: 0.6735673210012761 RNA: 0.7800958702253933
1.0 0.0 0.13999219 1.0 0.0 0.11338467
train/BRCA_8.pkl

===========> Training... <===========
train epoch 0: loss_ae: 0.079913 loss_mask_ae: 0.213403 loss_corr: 0.997659
train epoch 10: loss_ae: 0.060584 loss_mask_ae: 0.120158 loss_corr: 0.961184
train epoch 20: loss_ae: 0.050569 loss_mask_ae: 0.099348 loss_corr: 0.902109
train epoch 30: loss_ae: 0.048586 loss_mask_ae: 0.095377 loss_corr: 0.876824
train epoch 40: loss_ae: 0.046253 loss_mask_ae: 0.091611 loss_corr: 0.877667
train epoch 50: loss_ae: 0.041326 loss_mask_ae: 0.056664 loss_corr: 0.871530
train epoch 60: loss_ae: 0.038350 loss_mask_ae: 0.058586 loss_corr: 0.864259
train epoch 70: loss_ae: 0.038736 loss_mask_ae: 0.076597 loss_corr: 0.859641
train epoch 80: loss_ae: 0.038240 loss_mask_ae: 0.075022 loss_corr: 0.850833
train epoch 90: loss_ae: 0.035732 loss_mask_ae: 0.054896 loss_corr: 0.849034
Finished 8 Times Training and Testing...
(3798, 921) (45647, 921)
8.3757105 0.0 6.1131206 

C:\ProgramData\Anaconda3\envs\torch1.10\lib\site-packages\sklearn\utils\validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
C:\ProgramData\Anaconda3\envs\torch1.10\lib\site-packages\sklearn\utils\validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


ST: 0.6785185015794613 RNA: 0.7835552329338665
1.0 0.0 0.13808104 1.0 0.0 0.1121006
train/BRCA_9.pkl

===========> Training... <===========
train epoch 0: loss_ae: 0.079105 loss_mask_ae: 0.214153 loss_corr: 0.997699
train epoch 10: loss_ae: 0.062840 loss_mask_ae: 0.155482 loss_corr: 0.931539
train epoch 20: loss_ae: 0.054973 loss_mask_ae: 0.126566 loss_corr: 0.923705
train epoch 30: loss_ae: 0.049046 loss_mask_ae: 0.088776 loss_corr: 0.892152
train epoch 40: loss_ae: 0.047220 loss_mask_ae: 0.078958 loss_corr: 0.876901
train epoch 50: loss_ae: 0.043446 loss_mask_ae: 0.075487 loss_corr: 0.872055
train epoch 60: loss_ae: 0.040113 loss_mask_ae: 0.069214 loss_corr: 0.870668
train epoch 70: loss_ae: 0.042475 loss_mask_ae: 0.047241 loss_corr: 0.875449
train epoch 80: loss_ae: 0.038313 loss_mask_ae: 0.066235 loss_corr: 0.877425
train epoch 90: loss_ae: 0.036313 loss_mask_ae: 0.064111 loss_corr: 0.858517
Finished 9 Times Training and Testing...
